# Tensorflow Implementation

## Imports

We are importing all the neccessary libraries that we are gonna use.

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
# import seaborn as sns

/home/buckaroo/nlp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/buckaroo/nlp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/buckaroo/nlp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/buckaroo/nlp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

We are using [tensorflow hub](https://tfhub.dev/google/universal-sentence-encoder/4) which encodes text into high-dimensional vectors that can be used for text classification, semantic similarity, clustering and other natural language tasks. <br>

we are intializing the the model with this enocder below.

In [2]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
# module_url = "../data"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


Below in this small example , we are trying to compare the semantic similarity between these two sentences : <br>

Sentence 1 : `Python is a good language` <br>

Sentence 2 : `Language a good python is` <br>

As we know content-wise they contribute the same meaning. So we are expecting a cosine-angle score close to `1` 

In [3]:
sentences = ["what is your age ?","how old are you ?"]

similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
similarity_sentences_encodings = model(similarity_input_placeholder)

with tf.Session() as session:
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
  
    sentences_embeddings = session.run( similarity_sentences_encodings, feed_dict={
        similarity_input_placeholder: sentences
    })
  
    similarity = np.inner(sentences_embeddings[0], sentences_embeddings[1])
  
    print("Similarity is %s" % similarity)

Similarity is 0.80158687


# Importing Dataset

We are importing the dataset containing the key-value pair of Question and Answering and visualizing the contents.

In [4]:
df = pd.read_csv("../data/zeamed-web/zeamed-faq.csv")
df.head(5)

,Question,Answer
0,Does the Zeamed provide 24/7 customer support?,Yes
1,Is it a free service?,yes
2,Why is Zeamed providing a free service?,ZeaMed is absolutely free for consumers.Our a...
3,Where do I find zeamed doctor Profile on this ...,Select the test name and location and click on...
4,Will the patient info be secured?,Yes patient information/data is secured as per...


# Approach : 0

- For every questions in the `list` iterate over every question 
- Inilialize the Above loaded model with the `Question_{i}`
- And Calculate the cosine similarity between the `Question_{i}` and the `query`
     

In [ ]:
query = "who are you ?"

for question in df.Question:
    sentences = [ question , query ]

    similarity_input_placeholder = tf.placeholder(tf.string, shape=(None))
    similarity_sentences_encodings = model(similarity_input_placeholder)

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        session.run(tf.tables_initializer())
  
        sentences_embeddings = session.run( similarity_sentences_encodings, feed_dict={
            similarity_input_placeholder: sentences
        })
  
        similarity = np.inner(sentences_embeddings[0], sentences_embeddings[1])
  
        print("Similarity between %s & %s is %s" % (question , query , similarity) )

Similarity between Does the Zeamed provide 24/7 customer support? & who are you ? is -0.04937468
Similarity between Is it a free service? & who are you ? is 0.082572356
Similarity between Why is Zeamed providing a free service? & who are you ? is 0.035000674
Similarity between Where do I find zeamed doctor Profile on this app? & who are you ? is 0.089377984
Similarity between Will the patient info be secured?  & who are you ? is -0.072951436
Similarity between Who are ZeaMed’s users? & who are you ? is 0.5057231
Similarity between What is ZeaMed & who are you ? is 0.22169513
Similarity between If we book an appointment through you (Zeamed.com) do I stand a better chance of getting an appointment? & who are you ? is 0.04126934


# Approach : 1

- Inilialize the Above loaded model with the `Questions` as a constant
- And Calculate the cosine similarity between the `Questions` matrix and the `query`
- All at once , so we can minimise the runtime , computation power
     

In [ ]:
print(df.Question.shape)
shape = ( df.Question.shape[0] , 1 )
print(shape)

# questions_array = []
# for question in df.Question:
#    questions_array.append(question)
    
questions_array = np.array(df.Question).reshape(-1,1)

#print(questions_array)

#questions= tf.constant( questions_array , dtype=tf.string  ) 
query = tf.placeholder( tf.string )
#embed_questions = model(questions)
embed_query = model(query)

with tf.Session() as session:
    session.run( tf.global_variables_initializer() )
    session.run( tf.tables_initializer() )
    
    #question_matrix = session.run(embed_questions)
    query_matrix = session.run(embed_query , feed_dict = {
        query : ["who are you"]
    } )
    
    print( query_matrix )

In [ ]:
print(df.Question.shape)
shape = ( df.Question.shape[0] , 1 )
print(shape)

questions_array = []
for question in df.Question:
    questions_array.append(question)
    
#questions_array = np.array(df.Question).reshape(-1,1)

#print(questions_array)

#questions= tf.constant( questions_array , dtype=tf.string , shape = shape  ) 
questions = tf.placeholder( tf.string )
embed_questions = model( questions )

query = tf.placeholder( tf.string )
embed_query = model(query)

with tf.Session() as session:
    session.run( tf.global_variables_initializer() )
    session.run( tf.tables_initializer() )
    
    question_matrix , query_matrix = session.run( [ embed_questions ,embed_query ] , feed_dict = {
        questions : questions_array,
        query : ['what do you mean by zeamed']
    })

    
    print( question_matrix.shape , query_matrix.shape )
    product = np.inner(query_matrix , question_matrix)
    # product = product.reshape(-1,1)
    
    for question , score in zip(questions_array , product[0].tolist()):
        print( question , round(score,2)) 
    

In [ ]:
print(df.Question.shape)
shape = ( df.Question.shape[0] , 1 )
print(shape)

questions_array = []
for question in df.Question:
    questions_array.append(question)
    
#questions_array = np.array(df.Question).reshape(-1,1)

#print(questions_array)

#questions= tf.constant( questions_array , dtype=tf.string , shape = shape  ) 
questions = tf.placeholder( tf.string )
embed_questions = model( df.Question )

query = tf.placeholder( tf.string )
embed_query = model(query)

with tf.Session() as session:
    session.run( tf.global_variables_initializer() )
    session.run( tf.tables_initializer() )
    
    question_matrix , query_matrix = session.run( [ embed_questions ,embed_query ] , feed_dict = {
        # questions : questions_array,
        query : ['what is mean by zeamed ?']
    })

    
    print( question_matrix.shape , query_matrix.shape )
    product = np.inner(query_matrix , question_matrix)
    # product = product.reshape(-1,1)
    
    max_score = 1e-10
    choosen_question = "Please provide a valid response"
    for question , score in zip(questions_array , product[0].tolist()):
        
        # print( question , round(score,2)) 
        
        if ( round(score,2) > 0.50 and max_score < round(score,2) ):
            max_score = score
            choosen_question = question
            # print(question,max_score)
    if ( max_score == 1e-10 ):
        print( choosen_question , max_score )
    else:
        print('choosen question => ',choosen_question,max_score)
        choosen = df.Question == choosen_question
        print(df[choosen].Question.tolist()[0] )
        print(df[choosen].Answer.tolist()[0] )
        
    